# Tutorial 3: Cycled experiment


**Goal**: To run a cycled data assimilation experiment.
[`cycled_exp.py`](https://github.com/lkugler/DART-WRF/blob/master/generate_free.py) contains an example which will be explained here:

After configuring your experiment the loop looks like

```python
prior_path_exp = '/jetfs/home/lkugler/data/sim_archive/exp_v1.19_P2_noDA'

init_time = dt.datetime(2008, 7, 30, 13)
time = dt.datetime(2008, 7, 30, 14)
last_assim_time = dt.datetime(2008, 7, 30, 14)
forecast_until = dt.datetime(2008, 7, 30, 14, 15)

prepare_WRFrundir(init_time)
id = run_ideal(depends_on=id)

prior_init_time = init_time
prior_valid_time = time

while time <= last_assim_time:

    # usually we take the prior from the current time
    # but one could use a prior from a different time from another run
    # i.e. 13z as a prior to assimilate 12z observations
    prior_valid_time = time

    id = assimilate(time, prior_init_time, prior_valid_time, prior_path_exp, depends_on=id)

    # 1) Set posterior = prior
    id = prepare_IC_from_prior(prior_path_exp, prior_init_time, prior_valid_time, depends_on=id)

    # 2) Update posterior += updates from assimilation
    id = update_IC_from_DA(time, depends_on=id)

    # How long shall we integrate?
    timedelta_integrate = timedelta_btw_assim
    output_restart_interval = timedelta_btw_assim.total_seconds()/60
    if time == last_assim_time: #this_forecast_init.minute in [0,]:  # longer forecast every full hour
        timedelta_integrate = forecast_until - last_assim_time  # dt.timedelta(hours=4)
        output_restart_interval = 9999  # no restart file after last assim

    # 3) Run WRF ensemble
    id = run_ENS(begin=time,  # start integration from here
                end=time + timedelta_integrate,  # integrate until here
                output_restart_interval=output_restart_interval,
                depends_on=id)

    # as we have WRF output, we can use own exp path as prior
    prior_path_exp = cluster.archivedir       

    id_sat = create_satimages(time, depends_on=id)

    # increment time
    time += timedelta_btw_assim

    # update time variables
    prior_init_time = time - timedelta_btw_assim
        
verify_sat(id_sat)
verify_wrf(id)
verify_fast(id)
```

#### Job scheduling status
The script submits jobs into the SLURM queue with dependencies so that SLURM starts the jobs itself as soon as resources are available. Most jobs need only a few cores, but model integration is done across many nodes:
```bash
$ squeue -u `whoami` --sort=i
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1710274  mem_0384 prepwrfr  lkugler PD       0:00      1 (Priority)
           1710275  mem_0384 IC-prior  lkugler PD       0:00      1 (Dependency)
           1710276  mem_0384 Assim-42  lkugler PD       0:00      1 (Dependency)
           1710277  mem_0384 IC-prior  lkugler PD       0:00      1 (Dependency)
           1710278  mem_0384 IC-updat  lkugler PD       0:00      1 (Dependency)
           1710279  mem_0384 preWRF2-  lkugler PD       0:00      1 (Dependency)
    1710280_[1-10]  mem_0384 runWRF2-  lkugler PD       0:00      1 (Dependency)
           1710281  mem_0384 pRTTOV-6  lkugler PD       0:00      1 (Dependency)
           1710282  mem_0384 Assim-3a  lkugler PD       0:00      1 (Dependency)
           1710283  mem_0384 IC-prior  lkugler PD       0:00      1 (Dependency)
           1710284  mem_0384 IC-updat  lkugler PD       0:00      1 (Dependency)
           1710285  mem_0384 preWRF2-  lkugler PD       0:00      1 (Dependency)
    1710286_[1-10]  mem_0384 runWRF2-  lkugler PD       0:00      1 (Dependency)
           1710287  mem_0384 pRTTOV-7  lkugler PD       0:00      1 (Dependency)
```

